In [1]:
%load_ext autoreload
%autoreload 2
import os 
import pandas as pd
import numpy as np
os.chdir('..')
from src.data.make_dataset import get_preprocessed_datasets
from src.features.build_features import FeatureEngineering

In [2]:
df_bitcoin, df, df_test = get_preprocessed_datasets()

2019-11-03 19:32:56,917 - src.data.make_dataset - INFO - making final data sets from raw data
2019-11-03 19:32:57,177 - src.data.make_dataset - INFO - Shape of df_bitcoin: (2355, 2)
2019-11-03 19:32:57,178 - src.data.make_dataset - INFO - Shape of df: (4757, 91)


In [3]:
fe = FeatureEngineering(df, df_bitcoin, df_test)
fe.construct_features()
fe.df_features.head()

2019-11-03 19:33:11,943 - src.features.build_features - INFO - Start filling NA values in transaction_count
2019-11-03 19:33:11,951 - src.features.build_features - INFO - Found 2695 NA values in column transaction_count
2019-11-03 19:33:11,953 - src.features.build_features - INFO - Filled NA values
2019-11-03 19:33:11,971 - src.features.build_features - INFO - Add function holder count
2019-11-03 19:33:11,982 - src.features.build_features - INFO - Start filling NA values in holder_count
2019-11-03 19:33:11,995 - src.features.build_features - INFO - Found 2699 NA values in column holder_count
2019-11-03 19:33:11,997 - src.features.build_features - INFO - Filled NA values
2019-11-03 19:33:12,013 - src.features.build_features - INFO - Adding categories_0


,OBS_ID,success,transaction_count,holder_count,labels_categories_0
0,235691,0,48262.141038,13576.305982,23
1,235692,1,975773.000000,109840.000000,29
2,235693,0,22730.000000,13914.000000,29
3,235694,0,125.000000,51.000000,26
4,235695,0,16417.000000,6250.000000,26


In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


In [30]:
X_train, y_train, X_test = fe.get_X_y()

X_train = X_train.values
#y_train = y_train.values

In [24]:
X_train

array([[4.8262141e+04, 1.3576306e+04, 2.3000000e+01],
       [9.7577300e+05, 1.0984000e+05, 2.9000000e+01],
       [2.2730000e+04, 1.3914000e+04, 2.9000000e+01],
       ...,
       [4.8262141e+04, 1.3576306e+04, 2.6000000e+01],
       [1.6800000e+02, 3.4000000e+01, 2.6000000e+01],
       [4.8262141e+04, 1.3576306e+04, 2.3000000e+01]])

In [25]:
X_train, X_val, y_train, y_val = train_test_split(
            X_train, y_train, test_size=0.2, random_state=42)

In [26]:
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

(3805, 3)
(3805,)
(952, 3)
(952,)


In [37]:
y_train = y_train.astype(int)

In [43]:
model = LogisticRegression(max_iter=1000)

In [44]:
model.fit(X_train, y_train)

/home/sandro/.local/share/virtualenvs/data_science_for_business_lmu-1wV0OWeh/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [45]:
y_preds = model.predict(X_val)

In [46]:
np.unique(y_preds)

array([0])

In [53]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=500)

In [54]:
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [55]:
y_preds = model.predict(X_val)

In [56]:
from sklearn.metrics import matthews_corrcoef


In [59]:
y_val = y_val.astype(int)

In [60]:
matthews_corrcoef(y_val, y_preds)

0.6492401042793636

## Test Set

In [63]:
X_train, y_train, X_test = fe.get_X_y()
X_train.head()

,transaction_count,holder_count,labels_categories_0
0,48262.141038,13576.305982,23
1,975773.000000,109840.000000,29
2,22730.000000,13914.000000,29
3,125.000000,51.000000,26
4,16417.000000,6250.000000,26


In [67]:
X_test_values = X_test[['transaction_count', 'holder_count', 'labels_categories_0']].values

In [69]:
preds_test = model.predict(X_test_values)

In [76]:
pd.DataFrame({"OBS_ID": X_test['OBS_ID'],
             "success": preds_test}).to_csv('data/processed/submission02', index=None)

In [78]:
pd.read_csv('data/processed/submission02')

,OBS_ID,success
0,1,0
1,2,0
2,3,0
3,4,1
4,5,0
...,...,...
996,997,0
997,998,0
998,999,0
999,1000,0
